In [1]:
#Load necessary libraries
from bioservices.kegg import KEGG
import pandas as pd
import progressbar
k = KEGG()

In [2]:
#Load the conversion table between affymetrix and common gene names
conversion_table = pd.read_csv('datasets/conversion_table.txt', delimiter='\t')


In [15]:
#Here, we create a concatination of all the pathways for all the genes in the dataset
file = open("pathways.txt", "w+")
pathway = {}
bar = progressbar.ProgressBar(maxval=conversion_table['gene_name'].size, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
i=0
j=0
for gene in conversion_table['gene_name']:
    temp_pathway = k.get_pathway_by_gene(gene, "hsa") # "hsa" assumes that all the genes corresponds to homo sapiens 
    if temp_pathway != None:
        for key in temp_pathway.keys():
            if key not in pathway.keys():
                file.write(str(j) + "\t" + str(key) +"\t"+ str(temp_pathway[key]+"\n"))
                j = j+1
    i = i+1
    bar.update(i)
bar.finish()
file.close() 

[========================================================================] 100%


In [67]:
#Load the conversion table between affymetrix and common gene names
pathways = pd.read_csv('pathways.txt', delimiter='\t')

In [69]:
new_pathways = pathways.drop_duplicates(['pathway_id', 'pathway_name'])
new_pathways

,id,pathway_id,pathway_name
0,0,hsa03030,DNA replication
1,1,hsa03420,Nucleotide excision repair
2,2,hsa03430,Mismatch repair
3,3,hsa03040,Spliceosome
4,4,hsa04010,MAPK signaling pathway
5,5,hsa04141,Protein processing in endoplasmic reticulum
6,6,hsa04144,Endocytosis
7,7,hsa04213,Longevity regulating pathway - multiple species
8,8,hsa04612,Antigen processing and presentation
9,9,hsa04915,Estrogen signaling pathway


In [9]:
#Create a file with a list of all the interrelations between the genes in a given pathway.
pfsnet_file = open("pfsnet_pathways.txt","w+")
pfsnet_file.write("pathway\tgene1\tgene2\n")
pfsnet_file.close()
pathways_edge = {}

In [17]:
#For all the distinct pathways find all the genes that build a given pathway. 
#Create a file with a list of all the interrelations between the genes in a given pathway.
pfsnet_file = open("pfsnet_pathways.txt","a")
bar = progressbar.ProgressBar(maxval=pathways.size, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
i=0
l=0
for j in pathways.index:
    ind = pathways.iloc[j]['pathway_id']

    try:
        network = k.parse_kgml_pathway(ind)
    except TypeError:
        print("Failed at index:" +str(j) +" corresponding to pathway: "+str(ind))
    else:
        entries = pd.DataFrame.from_dict(network['entries'])
        for item in network['relations']:
            gene1 = entries[entries['id']==item['entry1']]
            gene2 = entries[entries['id']==item['entry2']]
            if gene1['gene_names'].values != None and gene2['gene_names'].values != None:
                gene1 = gene1['gene_names'].values[0].split(','[0])[0]
                gene2 = gene2['gene_names'].values[0].split(','[0])[0]
                pfsnet_file.write(str(l) +"\t"+str(ind) +"\t"+ str(gene1)+"\t"+ str(gene2) + "\n")
                pathways_edge[k] = [str(ind), str(gene1), str(gene2)]
                l = l+1
                if l == 3000:
                    break
        i = i+1
        bar.update(i)
bar.finish()
pfsnet_file.close()

KeyboardInterrupt: 

In [32]:
pathways[pathways['pathway_id'] == 'hsa04010']

,id,pathway_id,pathway_name
4,4,hsa04010,MAPK signaling pathway
55,55,hsa04010,MAPK signaling pathway
918,918,hsa04010,MAPK signaling pathway
999,999,hsa04010,MAPK signaling pathway
1035,1035,hsa04010,MAPK signaling pathway
1047,1047,hsa04010,MAPK signaling pathway
1148,1148,hsa04010,MAPK signaling pathway
1249,1249,hsa04010,MAPK signaling pathway
1762,1762,hsa04010,MAPK signaling pathway
1766,1766,hsa04010,MAPK signaling pathway


In [33]:
pfsnet_pathways = pd.read_csv('pfsnet_pathways.txt', delimiter='\t')


In [34]:
pfsnet_pathways

,id,pathway,gene1,gene2
0,0,hsa04010,GNA12,RASA2
1,1,hsa04010,RAPGEF2,RRAS2
2,2,hsa04010,RASGRF1,RRAS2
3,3,hsa04010,RASGRP1,RRAS2
4,4,hsa04010,RAP1A,BRAF
5,5,hsa04010,TNF,TNFRSF1A
6,6,hsa04010,IL1A,IL1R1
7,7,hsa04010,FASLG,FAS
8,8,hsa04010,TGFB1,TGFBR1
9,9,hsa04010,FAS,DAXX


In [36]:
new_pathways = pathways.drop_duplicates(['pathway_id', 'pathway_name'])

In [46]:
new_pathways.index.size

330

In [48]:
#For all the distinct pathways find all the genes that build a given pathway. 
#Create a file with a list of all the interrelations between the genes in a given pathway.
pfsnet_file = open("pfsnet_pathways_new.txt","w+")
pfsnet_file.write("pathway\tgene1\tgene2\n")
bar = progressbar.ProgressBar(maxval=new_pathways.size, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
i=0
l=0
for j in range(new_pathways.index.size):
    ind = new_pathways.iloc[j]['pathway_id']

    try:
        network = k.parse_kgml_pathway(ind)
    except TypeError or IndexError:
        print("Failed at index:" +str(j) +" corresponding to pathway: "+str(ind))
    else:
        entries = pd.DataFrame.from_dict(network['entries'])
        for item in network['relations']:
            gene1 = entries[entries['id']==item['entry1']]
            gene2 = entries[entries['id']==item['entry2']]
            if gene1['gene_names'].values != None and gene2['gene_names'].values != None:
                gene1 = gene1['gene_names'].values[0].split(','[0])[0]
                gene2 = gene2['gene_names'].values[0].split(','[0])[0]
                pfsnet_file.write(str(l) +"\t"+str(ind) +"\t"+ str(gene1)+"\t"+ str(gene2) + "\n")
                pathways_edge[k] = [str(ind), str(gene1), str(gene2)]
                l = l+1
                if l == 3000:
                    break
        i = i+1
        bar.update(i)
bar.finish()
pfsnet_file.close()

[========================================================================] 100%


In [50]:
pfsnet_pathways_new = pd.read_csv('pfsnet_pathways_new.txt', delimiter='\t')
pfsnet_pathways_new_cleaned = pfsnet_pathways_new.drop_duplicates(['pathway', 'gene1', 'gene2'])

In [54]:
pfsnet_pathways_new_cleaned.to_csv('pfsnet_pathways_cleaned.txt', '\t')

In [66]:
pfsnet_pathways_new_cleaned

,pathway,gene1,gene2
0,hsa04010,GNA12,RASA2
1,hsa04010,RAPGEF2,RRAS2
2,hsa04010,RASGRF1,RRAS2
3,hsa04010,RASGRP1,RRAS2
4,hsa04010,RAP1A,BRAF
5,hsa04010,TNF,TNFRSF1A
6,hsa04010,IL1A,IL1R1
7,hsa04010,FASLG,FAS
8,hsa04010,TGFB1,TGFBR1
9,hsa04010,FAS,DAXX


In [64]:
gene_list = pd.concat([pfsnet_pathways_new_cleaned['gene1'], pfsnet_pathways_new_cleaned['gene2']])
gene_list = pd.DataFrame(gene_list, columns=['gene_name'])
gene_list = gene_list.drop_duplicates(['gene_name'])

In [65]:
gene_list

,gene_name
0,GNA12
1,RAPGEF2
2,RASGRF1
3,RASGRP1
4,RAP1A
5,TNF
6,IL1A
7,FASLG
8,TGFB1
9,FAS


In [79]:
temp_pathway = k.get_pathway_by_gene('CYP2A6', "hsa")

In [80]:
temp_pathway

{'hsa00232': 'Caffeine metabolism',
 'hsa00830': 'Retinol metabolism',
 'hsa00980': 'Metabolism of xenobiotics by cytochrome P450',
 'hsa00982': 'Drug metabolism - cytochrome P450',
 'hsa00983': 'Drug metabolism - other enzymes',
 'hsa01100': 'Metabolic pathways',
 'hsa05204': 'Chemical carcinogenesis'}